<a href="https://colab.research.google.com/github/apowell18/Sudoku/blob/main/Sudoku_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project 1: Suduko solver

In [ ]:
#!/usr/bin/env python
#coding:utf-8

"""
Each sudoku board is represented as a dictionary with string keys and
int values.
e.g. my_board['A1'] = 8
"""

ROW = "ABCDEFGHI"
COL = "123456789"


def print_board(board):
    """Helper function to print board in a square."""
    print("-----------------")
    for i in ROW:
        row = ''
        for j in COL:
            row += (str(board[i + j]) + " ")
        print(row)


def board_to_string(board):
    """Helper function to convert board dictionary to string for writing."""
    ordered_vals = []
    for r in ROW:
        for c in COL:
            ordered_vals.append(str(board[r + c]))
    return ''.join(ordered_vals)



In [ ]:
# checking if all spaces are completed
def isCompleted(b): 
  for r in range (0, 9): 
      for c in range(0,9): 
        num = b[ROW[r] + COL[c]]
        if num == 0: 
          return False
  return True

In [ ]:
#determines if the value is valid for the cell
def isValid(b,x, y, n): # check if valid 
    #check row
    num = 0
    for i in range (0, 9): 
      num = b[ROW[x] + COL[i]]
      if num == n and (y is not i):
        return False
    #check column
    for j in range (0, 9): 
      num = b[ROW[j] + COL[y]]
      if num == n and (x is not j):
        return False
    #check first 3x3 box
    for i in range (0, 3): 
      for j in range(0,3): 
        num = b[ROW[((x//3)*3)+i] + COL[((y//3)*3)+j]]
        if num == n and ((x , y) is not (i , j)): 
          return False
    return True

In [ ]:
#returns only one empty cell
def emptySquare(b): 
  cell = []
  for r in range (0, 9): 
      for c in range(0,9): 
        num = b[ROW[r] + COL[c]]
        if num == 0: 
          cell.append(r)
          cell.append(c)
  if len(cell) > 0:
    return cell
  return None

In [ ]:
#collecting list of all empty cells
def emptySquares(b): 
  emptyCells = []
  for r in range (0, 9): 
      for c in range(0,9): 
        num = b[ROW[r] + COL[c]]
        if num == 0: 
          emptyCells.append((r, c))
  if len(emptyCells) > 0: 
    return emptyCells
  return None

In [ ]:
#find all of the legal values in for the cell
def findLegals(b, row, col):
  legalCells = []
  for i in range(1, 10): 
    if isValid(b, row, col, i): 
      legalCells.append(i)
  return legalCells

In [ ]:
def backtrackingOld(board):
    """Takes a board and returns solved board."""
    #start only manipulate spaces with 0s
    
    backtrackHelperOld(board)
    solved_board = board
    emptySlot = emptySquare(solved_board)
    if emptySlot is None: #check if board is already completed 
     return solved_board
    
    #obtain the first cell
    a = emptySlot[0] #row
    b = emptySlot[1] #col
    

    num = findLegals(solved_board,a, b)

    for no in num: #iterate through valid values
        if isValid(solved_board,a, b, no): #check if valid
          solved_board[ROW[a] + COL[b]] = no  
          if backtrackingOld(solved_board): #backtrack
            return solved_board 
          solved_board[ROW[a] + COL[b]] = 0 #reset cell

    return board

In [ ]:
#Only returns board with cells with least value stored. 
def backtrackHelperOld(board):
    """Helps backtracking method by inserting single value cells"""
    #start only manipulate spaces with 0s
    solved_board = board
    emptySlots = emptySquares(solved_board)
    if emptySlots is None: #check if board is already completed 
     return solved_board
    eCells = emptySquares(solved_board) #find the empty boards
    #implement solver
    holdLegalCells = []
    for i in range(0, len(eCells)):
      row = eCells[i][0]
      col = eCells[i][1]
      temp = findLegals(solved_board, row , col) #find legal vals
      if len(temp) == 1: #if cell only has one value, insert
        solved_board[ROW[row] + COL[col]] = temp[0]
        if backtrackingOld(solved_board): #backtrack, safety net to catch invalid vals
          return solved_board
        solved_board[ROW[row] + COL[col]] = 0
    return solved_board

In [ ]:
def backtrackingUpdate(board):
    """Takes a board and returns solved board."""
    #start only manipulate spaces with 0s
    #solved_board = board
    if isCompleted(board): #check if board is already completed 
     return board
    
    #obtain coordinates and legal value
    cells = backtrackHelperNew(board) 
    
    a = cells[0][0][0] #row
    b = cells[0][0][1] #column
    #implement solver
    num = cells[0][1] #legal value
        
    for no in num: #iterate through valid values
      if isValid(board,a,b,no): #double check if still valid
        board[ROW[a] + COL[b]] = no 
        if backtrackingUpdate(board): #backtrack
          return board
        board[ROW[a] + COL[b]] = 0 #reset
    return None

In [ ]:
#Only returns cells with least value stored. 
def backtrackHelperNew(board):
    """Helps backtracking method by sorting and finding the cells with mrv"""
    #collect empty cells
    solved_board = board
    emptySlots = emptySquares(solved_board)
    eCells = emptySquares(solved_board) 
    
    #implement minimum remaining value collection
    holdLegalCells = []
    for i in range(0, len(eCells)):
      row = eCells[i][0] #row
      col = eCells[i][1]  #col
      temp = findLegals(solved_board, row , col) #find legal vals
      holdLegalCells.append(((row,col), temp ))
    return sorted(holdLegalCells, key=lambda pos: (len(pos[1]))) #sort by length of legal cells

In [ ]:
if __name__ == '__main__':
    #  Read boards from source.
    #src_filename = 'sudoku_boards.txt' #mount google drive
    src_filename = '/content/drive/MyDrive/AI/Sudoko/sudoku_boards.txt'
    try:
        srcfile = open(src_filename, "r")
        sudoku_list = srcfile.read()
    except:
        print("Error reading the sudoku file %s" % src_filename)
        exit()

    # Setup output file
    out_filename = 'output.txt'
    outfile = open(out_filename, "w")

    # Solve each board using backtracking
    for line in sudoku_list.split("\n"):

        if len(line) < 9:
            continue

        # Parse boards to dict representation, scanning board L to R, Up to Down
        board = { ROW[r] + COL[c]: int(line[9*r+c])
                  for r in range(9) for c in range(9)}

        # Print starting board. TODO: Comment this out when timing runs.
        print_board(board)
        
        # Solve with backtracking
        #solved_board = backtrackingOld(board) #Original version/attempt - prefill boards
        solved_board = backtrackingUpdate(board)  #run updated version
        
        #previous version completes only 13
        #updated solution completes all boards
      
        # Print solved board. TODO: Comment this out when timing runs.
        if isCompleted(solved_board): #viewing how many boards are actually completed
          print_board(solved_board)
        
        # Write board to file
        outfile.write(board_to_string(solved_board))
        outfile.write('\n')


    print("Finishing all boards in file.")

